In [1]:
import numpy as np

In [2]:
X_train = np.load("Fashion_MNIST/X_train.npy")
X_test = np.load("Fashion_MNIST/X_test.npy")
y_train = np.load("Fashion_MNIST/y_train.npy")
y_test = np.load("Fashion_MNIST/y_test.npy")

In [3]:
X_train.shape

(60000, 784)

In [4]:
X_train.mean()

72.94035223214286

In [5]:
X_train = X_train/255
X_test = X_test/255

In [6]:
y_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [7]:
from sklearn.preprocessing import OneHotEncoder

In [8]:
encoder = OneHotEncoder(sparse=False)

In [9]:
encoder.fit(y_train.reshape(-1,1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [10]:
y_train = encoder.transform(y_train.reshape(-1,1))#make column vector
y_test = encoder.transform(y_test.reshape(-1,1))

In [11]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [12]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

### Tensorflow

In [13]:
import tensorflow as tf

/Users/emma/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [14]:
tf.reset_default_graph()#reset the graph

In [15]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

In [16]:
W1 = tf.Variable(tf.random_normal(shape = [784, 256], stddev= 0.1))# the same as /100
b1 = tf.Variable(tf.zeros(shape=[256]))

In [17]:
W2 = tf.Variable(tf.random_normal(shape = [ 256, 10], stddev= 0.1))# the same as /100
b2 = tf.Variable(tf.zeros(shape=[10]))

In [18]:
Z1 = tf.matmul(X,W1) + b1
A1 = tf.nn.relu(Z1)

output = tf.matmul(A1, W2) + b2

In [19]:
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits=output)
cost = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_operation = optimizer.minimize(cost)

In [20]:
correct_predictions = tf.equal(tf.argmax(output, 1), tf.argmax(y,1))#inverse of one hot encoding 

In [ ]:
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("/Users/emma/ML/python_with_Ardalan.ipynb/Deep Learning/1")
    writer.add_graph(sess.graph)
    for epoch in range (1,101):
        for X_batch, y_batch in get_next_batch(X_train, y_train, size=64):
          _ =  sess.run([train_operation], {X:X_batch, y:y_batch})
        if epoch%10 == 0:
            acc, c = sess.run([accuracy, cost], {X:X_test, y:y_test})
            print ("Epoch", epoch, "Accuracy", acc, "Loss", c)

Epoch 10 Accuracy 0.854 Loss 0.44454572
Epoch 20 Accuracy 0.8655 Loss 0.43499842
